In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from typing import List, Tuple

In [10]:

NUM_CLUSTERS = 2
RANDOM_STATE = 42
MAX_ITERATIONS = 500

In [11]:

data = pd.read_csv("data_device.csv")

# Bỏ cột đầu tiên (giả sử là ID)
raw_points = data.iloc[:, 1:].values # Lấy tất cả các hàng, bắt đầu từ cột thứ 1

# Định nghĩa trọng số và Chỉ mục (Sử dụng Pandas/NumPy sau khi có dữ liệu)
FEATURE_INDEX = {
    "Total Ram": 0,
    "Storage": 1,
    "Internet": 2,
    "Core": 3
}

WEIGHTS = {
    "Core": 1.4,
    "Total Ram": 1.2,
    "Internet": 1.1,
    "Storage": 0.65
}



In [12]:
# --- Chuẩn hóa Min-Max bằng Sklearn ---
scaler = MinMaxScaler()
# Sử dụng fit_transform để vừa tính min/max, vừa chuẩn hóa
points_normalized = scaler.fit_transform(raw_points)

# --- Gán trọng số thủ công ---
points_weighted = points_normalized.copy()

for feat, w in WEIGHTS.items():
    # Nhân trọng số trực tiếp vào cột tương ứng
    points_weighted[:, FEATURE_INDEX[feat]] *= w

# Dữ liệu đã sẵn sàng cho K-Means
points_final = points_weighted

In [13]:
# Tạo mô hình KMeans
kmeans_model = KMeans(
    n_clusters=NUM_CLUSTERS,            # Số lượng cụm
    init='k-means++',                   # Khởi tạo tối ưu (K-Means++)
    n_init='auto',                      # Số lần chạy thuật toán với các khởi tạo khác nhau
    max_iter=MAX_ITERATIONS,            # Số lần lặp tối đa
    random_state=RANDOM_STATE,          # Đảm bảo kết quả lặp lại
    algorithm='lloyd'                   # Thuật toán K-Means cổ điển (hiệu quả)
)

# Huấn luyện mô hình trên dữ liệu đã được gán trọng số
kmeans_model.fit(points_final)

# Lấy kết quả
final_centers_weighted = kmeans_model.cluster_centers_  # Tâm cụm trong không gian đã gán trọng số
cluster_labels = kmeans_model.labels_                   # Nhãn cụm (0 hoặc 1) cho mỗi điểm

In [14]:
from scipy.spatial.distance import cdist

def nearest_raw_point(raw_points: np.ndarray,
                      target_centers: np.ndarray) -> np.ndarray:
    """
    Tìm điểm dữ liệu thô gần nhất với mỗi tâm cụm.
    Sử dụng cdist để tính khoảng cách ma trận nhanh chóng.

    Args:
        raw_points: Dữ liệu THÔ gốc (chưa chuẩn hóa/trọng số).
        target_centers: Các tâm cụm ĐÃ HỘI TỤ (trong không gian đã gán trọng số).
                        Tuy nhiên, chúng ta cần tìm điểm gần nhất trong KHÔNG GIAN ĐÃ TRỌNG SỐ.

    Returns:
        Một mảng NumPy chứa các điểm thô (chưa chuẩn hóa) gần nhất.
    """

    # --- QUAN TRỌNG: Phải tìm điểm gần nhất trong KHÔNG GIAN ĐÃ TRỌNG SỐ ---
    # raw_points là thô, target_centers là trong không gian (Normalized + Weighted)
    # Ta phải chuẩn hóa và gán trọng số cho raw_points để so sánh

    # 1. Chuẩn hóa lại raw_points (đã có scaler)
    points_scaled_recheck = scaler.transform(raw_points)

    # 2. Gán trọng số lại cho raw_points
    points_for_comparison = points_scaled_recheck.copy()
    for feat, w in WEIGHTS.items():
        points_for_comparison[:, FEATURE_INDEX[feat]] *= w

    final_raw_centers = []

    for i, center in enumerate(target_centers):
        # Tính khoảng cách Euclidean giữa tâm cụm này (center) và TẤT CẢ các điểm dữ liệu
        # trong không gian đã gán trọng số (points_for_comparison)

        # cdist(center.reshape(1, -1), points_for_comparison) trả về mảng khoảng cách (1, N)
        distances = cdist(center.reshape(1, -1), points_for_comparison, metric='euclidean')

        # Tìm chỉ mục của điểm có khoảng cách nhỏ nhất
        closest_index = np.argmin(distances)

        # Lấy điểm THÔ tương ứng
        closest_raw_point = raw_points[closest_index]
        final_raw_centers.append(closest_raw_point)

    return np.array(final_raw_centers)

# Chạy hàm để lấy kết quả cuối cùng
final_centers = nearest_raw_point(raw_points, final_centers_weighted)
